In [1]:
import numpy as np
import pandas as pd
from collections import Counter
from itertools import combinations

In [2]:
movies = pd.read_csv('movie_bd_v5.csv')
movies.sample(5)


,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
359,tt0458525,150000000,341131793,X-Men Origins: Wolverine,Hugh Jackman|Liev Schreiber|Danny Huston|Lynn ...,Gavin Hood,Witness the Origin.,"After seeking to live a normal life, Logan set...",107,Adventure|Action|Thriller|Science Fiction,Ingenious Film Partners|Twentieth Century Fox ...,4/28/2009,6.2,2009
11,tt1617661,176000003,183987723,Jupiter Ascending,Mila Kunis|Channing Tatum|Sean Bean|Eddie Redm...,Lana Wachowski|Lilly Wachowski,Expand your universe.,In a universe where human genetic material is ...,124,Science Fiction|Fantasy|Action|Adventure,Village Roadshow Pictures|Dune Entertainment|A...,2/4/2015,5.2,2015
1303,tt1869716,6500000,2401510,The East,Brit Marling|Alexander SkarsgÃ¥rd|Ellen Page|T...,Zal Batmanglij,"Spy on us, we'll spy on you.",An operative for an elite private intelligence...,116,Drama|Thriller,Scott Free Productions,5/31/2013,6.3,2013
363,tt1232783,12500000,27206120,Sorority Row,Rumer Willis|Briana Evigan|Jamie Chung|Leah Pi...,Stewart Hendler,The Sisters of Theta Pi Are Dying To Keep a Se...,When five sorority girls inadvertently cause t...,101,Horror|Mystery|Thriller,Summit Entertainment|Karz Entertainment,9/9/2009,5.3,2009
1575,tt0298148,150000000,919838758,Shrek 2,Mike Myers|Eddie Murphy|Cameron Diaz|Julie And...,Andrew Adamson|Kelly Asbury|Conrad Vernon,Once upon another time...,"Shrek, Fiona and Donkey set off to Far, Far Aw...",93,Adventure|Animation|Comedy|Family|Fantasy,DreamWorks SKG|Pacific Data Images (PDI)|Dream...,5/19/2004,6.5,2004


In [3]:
movies.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


In [4]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   imdb_id               1889 non-null   object 
 1   budget                1889 non-null   int64  
 2   revenue               1889 non-null   int64  
 3   original_title        1889 non-null   object 
 4   cast                  1889 non-null   object 
 5   director              1889 non-null   object 
 6   tagline               1889 non-null   object 
 7   overview              1889 non-null   object 
 8   runtime               1889 non-null   int64  
 9   genres                1889 non-null   object 
 10  production_companies  1889 non-null   object 
 11  release_date          1889 non-null   object 
 12  vote_average          1889 non-null   float64
 13  release_year          1889 non-null   int64  
dtypes: float64(1), int64(4), object(9)
memory usage: 206.7+ KB


Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

# Предобработка

In [5]:
answers = {} # create a dictionary for answers


# To check what films are in Top 1% of the list (question #26),
# define list-comparing function:

def check_if_in(small_list, big_list):
    if [x for x in small_list if x not in big_list] == []:
        return True
    return False


# We'll need profit values for some questions,
# add a column where "revenue minus budget" 
# values are stored for every movie.
# To be placed right after the "budget" column for a better visibility:

movies = pd.read_csv('movie_bd_v5.csv')
movies.insert(3, 'profit', 
              movies.apply(lambda x: x.revenue - x.budget, axis=1))


# To analyze seasons and months, 
# also add a column 
# with numeric values for the month of a movie's release 
# + dictionary for better visibility:

movies['release_month'] = movies.apply(lambda x: x.release_date.split('/')[0], 
                                       axis=1)

month_dict={'1':'Январь',
           '2':'Февраль', 
           '3':'Март',
           '4':'Апрель',
           '5':'Май',
           '6':'Июнь',
           '7':'Июль',
           '8':'Август',
           '9':'Сентябрь',
           '10':'Октябрь',
           '11':'Ноябрь',
           '12':'Декабрь'
           }


# Title length (in symbols) is studied in question #24:

movies.insert(5, 'title_length', 
              movies.apply(lambda x: len(x.original_title), axis=1))


# Overview length (in words) is studied in question #25:

movies.insert(10, 'overview_length', 
              movies.apply(lambda x: len(x.overview.split()), axis=1))


movies.sample(5)

,imdb_id,budget,revenue,profit,original_title,title_length,cast,director,tagline,overview,overview_length,runtime,genres,production_companies,release_date,vote_average,release_year,release_month
1228,tt1535109,55000000,95000000,40000000,Captain Phillips,16,Tom Hanks|Catherine Keener|Max Martini|Chris M...,Paul Greengrass,Out here survival is everything.,The true story of Captain Richard Phillips and...,32,134,Action|Drama|Thriller,Columbia Pictures,10/10/2013,7.7,2013,10
111,tt1243974,37000000,26250020,-10749980,Aloha,5,Bradley Cooper|Emma Stone|Rachel McAdams|John ...,Cameron Crowe,Sometimes you have to say goodbye before you c...,A celebrated military contractor returns to th...,31,105,Adventure|Comedy|Romance,Columbia Pictures|Vinyl Films|Regency Enterpri...,5/27/2015,5.2,2015,5
1772,tt0762114,35000000,69307224,34307224,License to Wed,14,Robin Williams|Mandy Moore|John Krasinski|Eric...,Ken Kwapis,First came love... then came Reverend Frank.,"Newly engaged, Ben and Sadie can't wait to sta...",49,91,Comedy,Village Roadshow Pictures|Warner Bros. Picture...,7/4/2007,5.1,2007,7
637,tt1068680,70000000,225990978,155990978,Yes Man,7,Jim Carrey|Zooey Deschanel|Rhys Darby|Sasha Al...,Peyton Reed,One Word Can Change Everything.,Carl Allen has stumbled across a way to shake ...,24,104,Comedy|Romance,The Zanuck Company|Heyday films|Warner Bros.,12/9/2008,6.3,2008,12
1404,tt0383222,25000000,2405420,-22594580,BloodRayne,10,Kristanna Loken|Ben Kingsley|Michelle Rodrigue...,Uwe Boll,Revenge never tasted so sweet.,"In eighteenth century Romania, Rayne, a dhampi...",52,95,Action|Adventure|Fantasy|Horror,Boll KG,10/22/2005,3.8,2005,10


# 1. У какого фильма из списка самый большой бюджет?

In [6]:
# answer
answers['1'] = '5. Pirates of the Caribbean: On Stranger Tides (tt1298650)'
# +

In [7]:
# solution code
movies[movies.budget == movies.budget.max()][['original_title', 'imdb_id']]

,original_title,imdb_id
723,Pirates of the Caribbean: On Stranger Tides,tt1298650


# 2. Какой из фильмов самый длительный (в минутах)?

In [8]:
# answer
answers['2'] = '2. Gods and Generals (tt0279111)'
# +

In [9]:
# solution code
movies[movies.runtime == movies.runtime.max()][['original_title', 
                                                'imdb_id']]

,original_title,imdb_id
1157,Gods and Generals,tt0279111


# 3. Какой из фильмов самый короткий (в минутах)?





In [10]:
# answer
answers['3'] = '3. Winnie the Pooh (tt1449283)'
# +

In [11]:
# solution code
movies[movies.runtime == movies.runtime.min()][['original_title', 
                                                'imdb_id']]

,original_title,imdb_id
768,Winnie the Pooh,tt1449283


# 4. Какова средняя длительность фильмов?


In [12]:
# answer
answers['4'] = '2. 110'
# +

In [13]:
# solution code
round(movies.runtime.mean())

110

# 5. Каково медианное значение длительности фильмов? 

In [14]:
# ответ
answers['5'] = '1. 107'
# +

In [15]:
# solution code
round(movies.runtime.median())

107

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [16]:
# answer
answers['6'] = '5. Avatar (tt0499549)'
# +

In [17]:
# solution code
movies[movies.profit == movies.profit.max()][['original_title', 
                                              'imdb_id']]

,original_title,imdb_id
239,Avatar,tt0499549


# 7. Какой фильм самый убыточный? 

In [18]:
# answer
answers['7'] = '5. The Lone Ranger (tt1210819)'
# +

In [19]:
# solution code
movies[movies.profit == movies.profit.min()][['original_title', 
                                              'imdb_id']]

,original_title,imdb_id
1245,The Lone Ranger,tt1210819


# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [20]:
# answer
answers['8'] = '1. 1478'
# +

In [21]:
# solution code
movies[movies.profit > 0].imdb_id.count()

1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [22]:
# answer
answers['9'] = '4. The Dark Knight (tt0468569)'
# +

In [23]:
# solution code
movies_2008 = movies[movies.release_year == 2008].copy()
movies_2008[movies_2008.revenue == movies_2008.revenue.max()][['original_title', 
                                                               'imdb_id']]

,original_title,imdb_id
599,The Dark Knight,tt0468569


# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [24]:
# answer
answers['10'] = '5. The Lone Ranger (tt1210819)'
# +

In [25]:
# solution code
movies_2012_2014 = movies[(movies.release_year >= 2012) &
                         (movies.release_year <= 2014)].copy()
movies_2012_2014[movies_2012_2014.profit == movies_2012_2014.profit.min()][['original_title', 
                                                                            'imdb_id']]

,original_title,imdb_id
1245,The Lone Ranger,tt1210819


# 11. Какого жанра фильмов больше всего?

In [26]:
# answer
answers['11'] = '3. Drama'
# +


# solution code
pd.Series(movies.genres.str.cat(sep='|').split('|')).value_counts().index[0]

'Drama'

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [27]:
# answer
answers['12'] = '1. Drama'
# +


# solution code
movies_profitable = movies[movies.profit > 0].copy()
pd.Series(movies_profitable.genres.str.cat(sep='|').split('|')).value_counts().index[0]

'Drama'

# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [28]:
# answer
answers['13'] = '5. Peter Jackson'
# +


# solution code
directors = pd.Series(movies.director.str.cat(sep='|').split('|')).unique()
revenue = pd.Series(index = directors, dtype = 'float64')

for director in directors:
    revenue[director] = movies.revenue[movies.director.map(lambda x: True if director in x else False)].sum()

revenue.idxmax()

'Peter Jackson'

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [29]:
# answer
answers['14'] = '3. Robert Rodriguez'
# +


# solution code #1
actions = movies[movies.genres.str.contains('Action') == True]  
action_directors = Counter(actions.director.str.cat(sep='|').split('|'))
action_directors.most_common(1)[0][0]

'Robert Rodriguez'

In [30]:
# solution code #2, very ineffective
directors = pd.Series(movies.director.str.cat(sep='|').split('|')).unique()
action_directors = pd.Series(index = directors, dtype = 'int64')

for director in directors:
    for movie in movies.index:
        if director in movies['director'][movie] and 'Action' in movies['genres'][movie]:
            action_directors[director] += 1

display(action_directors.idxmax())

'Robert Rodriguez'

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [31]:
# answer
answers['15'] = '3. Chris Hemsworth'
# +


# solution code
movies_2012 = movies[movies.release_year == 2012].copy()
movies_2012['actor'] = movies_2012.cast.str.split('|')
movies_2012.explode('actor').groupby(['actor']).sum().revenue.idxmax()

'Chris Hemsworth'

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [32]:
# answer
answers['16'] = '3. Matt Damon'
# +


# solution code
a_movies = movies[movies.budget > movies.budget.mean()].copy()
a_movies['actor'] = a_movies.cast.str.split('|')
a_movies.explode('actor').groupby(['actor']).count().idxmax()[0]

'Matt Damon'

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [33]:
# answer
answers['17'] = '2. Action'
# +


# solution code
cage_movies = movies[movies.cast.str.contains('Nicolas Cage') == True].copy()
pd.Series(cage_movies['genres'].str.cat(sep='|').split('|')).value_counts().index[0]

'Action'

# 18. Самый убыточный фильм от Paramount Pictures

In [34]:
# answer
answers['18'] = '1. K-19: The Widowmaker (tt0267626)'
# +


# solution code
pp_movies = movies[movies.production_companies.str.contains('Paramount Pictures') == True].copy()
pp_movies[pp_movies.profit == pp_movies.profit.min()][['original_title', 'imdb_id']]

,original_title,imdb_id
925,K-19: The Widowmaker,tt0267626


# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [35]:
# answer
answers['19'] = '5. 2015'
# +


# solution code
movies.groupby(['release_year']).sum().revenue.idxmax()

2015

# 20. Какой самый прибыльный год для студии Warner Bros?

In [36]:
# answer
answers['20'] = '1. 2014'
# +


# solution code
wb_movies = movies[movies.production_companies.str.contains('Warner Bros') == True]
wb_movies.groupby(['release_year']).sum().profit.idxmax()

2014

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [37]:
# answer
answers['21'] = '4. Сентябрь'
# +


# solution code
max_month = movies.groupby(['release_month']).count().idxmax()[0]
month_dict[max_month]

'Сентябрь'

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [38]:
# answer
answers['22'] = '2. 450'
# +


# solution code
movies.groupby(['release_month']).imdb_id.count().loc[['6', '7', '8']].sum()

450

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [39]:
# answer
answers['23'] = '5. Peter Jackson'
# +


# solution code
winter_movies = movies[movies.release_month.isin(['12', '1', '2'])]  
winter_directors = Counter(winter_movies.director.str.cat(sep='|').split('|'))
winter_directors.most_common(1)[0][0]

'Peter Jackson'

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [40]:
# answer
answers['24'] = '5. Four By Two Productions'
# +


# solution code
movies['company'] = movies.production_companies.str.split('|')
movies.explode('company').groupby(['company']).mean().title_length.idxmax()

'Four By Two Productions'

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [41]:
# answer
answers['25'] = '3. Midnight Picture Show'
# +


# solution code
movies['company'] = movies.production_companies.str.split('|')
movies.explode('company').groupby(['company']).mean().overview_length.idxmax()

'Midnight Picture Show'

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [42]:
# answer
answers['26'] = '1. Inside Out, The Dark Knight, 12 Years a Slave'
# +


# solution code
bm = movies[movies.vote_average > movies.vote_average.quantile(0.99)]['original_title'].values
display(check_if_in(
    'Inside Out, The Dark Knight, 12 Years a Slave'.split(', '), bm))
display(check_if_in(
    'BloodRayne, The Adventures of Rocky & Bullwinkle'.split(', '), bm))
display(check_if_in(
    'Batman Begins, The Lord of the Rings: The Return of the King, Upside Down'.split(', '), bm))
display(check_if_in(
    '300, Lucky Number Slevin, Kill Bill: Vol. 1'.split(', '), bm))
display(check_if_in(
    'Upside Down, Inside Out, Iron Man'.split(', '), bm))

True

False

False

False

False

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [43]:
# answer
answers['27'] = '5. Daniel Radcliffe & Rupert Grint'
# +


# solution code
combs = [] 
for cast in movies.cast.str.split('|'):
    combs.extend(combinations(cast, 2))
pd.Series(combs).value_counts()

#movies.cast.str.split('|')

(Daniel Radcliffe, Emma Watson)        8
(Daniel Radcliffe, Rupert Grint)       8
(Rupert Grint, Emma Watson)            7
(Ben Stiller, Owen Wilson)             6
(Johnny Depp, Helena Bonham Carter)    6
                                      ..
(Zooey Deschanel, Jon Heder)           1
(Sean Connery, Busta Rhymes)           1
(Charlie Sheen, Erica Ash)             1
(Jesse Bradford, Eliza Dushku)         1
(Leslie Mann, Allison Miller)          1
Length: 18121, dtype: int64

# Submission

In [44]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': '5. Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': '2. Gods and Generals (tt0279111)',
 '3': '3. Winnie the Pooh (tt1449283)',
 '4': '2. 110',
 '5': '1. 107',
 '6': '5. Avatar (tt0499549)',
 '7': '5. The Lone Ranger (tt1210819)',
 '8': '1. 1478',
 '9': '4. The Dark Knight (tt0468569)',
 '10': '5. The Lone Ranger (tt1210819)',
 '11': '3. Drama',
 '12': '1. Drama',
 '13': '5. Peter Jackson',
 '14': '3. Robert Rodriguez',
 '15': '3. Chris Hemsworth',
 '16': '3. Matt Damon',
 '17': '2. Action',
 '18': '1. K-19: The Widowmaker (tt0267626)',
 '19': '5. 2015',
 '20': '1. 2014',
 '21': '4. Сентябрь',
 '22': '2. 450',
 '23': '5. Peter Jackson',
 '24': '5. Four By Two Productions',
 '25': '3. Midnight Picture Show',
 '26': '1. Inside Out, The Dark Knight, 12 Years a Slave',
 '27': '5. Daniel Radcliffe & Rupert Grint'}

In [45]:
# и убедиться что ни чего не пропустил)
len(answers)

27